In [53]:
import numpy as np
import pandas as pd
import os
from torchvision import datasets, transforms
import torch
from PIL import Image

In [54]:
base_path = f"{os.path.expanduser('~')}/Downloads/"
if not os.path.exists(base_path):
    os.makedirs(base_path)

is_transform=True

In [55]:
def unpickle(file):
    import pickle

    with open(file, "rb") as fo:
        dict = pickle.load(fo, encoding="bytes")
    return dict


transform = transforms.Compose(
    [transforms.ToTensor(),
     ]) if not is_transform else transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
)
scale = 255 if is_transform else 1

# construct test set
test_dict = unpickle(base_path + "cifar-10-batches-py/test_batch")


data = test_dict[b"data"]
label = test_dict[b"labels"]

# from https://stackoverflow.com/questions/55319949/pil-typeerror-cannot-handle-this-data-type
img = np.array(data).astype(np.uint8).reshape(-1, 3, 32, 32).transpose((0, 2, 3, 1))

converted_data = []
for i in img:
    converted_i = Image.fromarray(i)
    converted_i = (transform(converted_i)*scale).type(torch.int32).reshape(3072)
    converted_data.append(converted_i)

data = np.stack(converted_data)
label = np.array(label).reshape((len(label), 1))

test_data = np.concatenate((data, label), axis=1)
print(f"Test shape: {test_data.shape}")
print(f"Test: {test_data[0]}")


test_df = pd.DataFrame(test_data)
# test_df = test_df.sample(frac=1)
# test_df.to_csv(base_path + "cifar10_test.csv", index=False, header=True)

# construct training set
batch_list = []
for i in range(1, 6):
    test_dict = unpickle(f"{base_path}cifar-10-batches-py/data_batch_{i}")

    data = test_dict[b"data"]
    label = test_dict[b"labels"]

    # from https://stackoverflow.com/questions/55319949/pil-typeerror-cannot-handle-this-data-type
    img = np.array(data).astype(np.uint8).reshape(-1, 3, 32, 32).transpose((0, 2, 3, 1))

    converted_batch = []
    for i in img:
        converted_i = Image.fromarray(i)
        converted_i = (transform(converted_i)*scale).type(torch.int32).reshape(3072)
        converted_batch.append(converted_i)

    data = np.stack(converted_batch)
    data = np.array(data)
    label = np.array(label).reshape((len(label), 1))

    merge_data = np.concatenate((data, label), axis=1)

    batch_list.append(merge_data)

train_data = np.concatenate(batch_list)
print(f"Train shape: {train_data.shape}")
print(f"Train: {train_data[0]}")

train_df = pd.DataFrame(train_data)
# train_df = train_df.sample(frac=1)
# train_df.to_csv(base_path + "cifar10_train.csv", index=False, header=True)


Test shape: (10000, 3073)
Test: [ 61  63  75 ...   3 -34   3]
Train shape: (50000, 3073)
Train: [-137 -169 -155 ...  -86 -110    6]


In [56]:
import numpy as np
import pandas as pd
import os
from torchvision import datasets, transforms
import torch
from PIL import Image

In [81]:
# transform = transforms.Compose(
#     [transforms.ToTensor(),
#      ])
torch.set_printoptions(precision=10)
batch_size = 32

path = f'{os.path.expanduser("~")}/Downloads'
trainset = datasets.CIFAR10(root=path, train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=False)

testset = datasets.CIFAR10(root=path, train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

for i, (img, label) in enumerate(trainloader):
    print(img.dtype)
    print(img[0, 0, 1, -3], (img*255)[0, 0, 1, -3], (img*255).type(torch.int)[0, 0, 1, -3], (torch.round(img*255).type(torch.int)/255.0)[0, 0, 1, -3])
    assert torch.allclose(img, torch.round(img*255).type(torch.int)/255.0, atol=1e-3)
    print((img[0]*255).type(torch.int), label[0])
    break

Files already downloaded and verified
Files already downloaded and verified
torch.float32
tensor(-0.0352941155) tensor(-8.9999990463) tensor(-8, dtype=torch.int32) tensor(-0.0352941193)
tensor([[[-137, -169, -155,  ...,   61,   49,   41],
         [-223, -255, -219,  ...,   -8,  -16,  -10],
         [-205, -223, -157,  ...,  -18,  -14,  -36],
         ...,
         [ 161,  147,  141,  ...,   65, -143, -149],
         [ 105,   91,  117,  ...,  113,  -60,  -88],
         [  99,   81,  103,  ...,  177,   47,   -8]],

        [[-131, -163, -159,  ...,    9,   -4,   -6],
         [-215, -255, -239,  ...,  -78,  -88,  -80],
         [-207, -241, -201,  ...,  -86,  -86, -108],
         ...,
         [  85,   51,   67,  ...,   11, -193, -187],
         [  23,   -8,   33,  ...,   41, -131, -149],
         [  33,    3,   29,  ...,  113,  -18,  -70]],

        [[-129, -165, -169,  ...,  -38,  -50,  -48],
         [-215, -255, -255,  ..., -145, -155, -141],
         [-213, -255, -239,  ..., -155, 

In [61]:
for i, (img, label) in enumerate(testloader):
    assert torch.equal(label, torch.tensor(test_data[i*batch_size:(i+1)*batch_size, -1]))
    # print(img.reshape(-1, 32*32*3).to(torch.float64))
    # print(torch.tensor(test_data[i*batch_size:(i+1)*batch_size, :-1]))
    # print(torch.allclose(img.reshape(-1, 32*32*3).to(torch.float64), torch.tensor(test_data[i*batch_size:(i+1)*batch_size, :-1])))
    assert torch.allclose(img.reshape(-1, 32*32*3), torch.tensor(test_data[i*batch_size:(i+1)*batch_size, :-1]))
    # assert label == torch.tensor(test_data[i*batch_size:(i+1)*batch_size, -1])

for i, (img, label) in enumerate(trainloader):
    assert torch.equal(label, torch.tensor(train_data[i*batch_size:(i+1)*batch_size, -1]))
    assert torch.equal(img.reshape(-1, 32*32*3).to(torch.float64), torch.tensor(train_data[i*batch_size:(i+1)*batch_size, :-1]))

    # assert label == torch.tensor(test_data[i*batch_size:(i+1)*batch_size, -1])

RuntimeError: Float did not match Long

## Validate outputs during training

In [167]:
import torch
import os
plaintext_outputs = torch.load(os.path.expanduser("~")+ "/DNN/output/plaintext-log.pt")
# plaintext_fc1_delta = torch.load(os.path.expanduser("~")+ "/DNN/output/plaintext-fc1-delta.pt")
print(plaintext_outputs[0].shape)
# print(plaintext_fc1_delta.shape)

iter = 16040
batch_size = 32

secure_outputs = []
# load secure training
with open(f"{os.path.expanduser('~')}/DNN/output/AlexNet preloaded init weights train_SPDZ_GPU_Full_CIFAR10_Tran_32_4_CE_debug_{iter}.txt", 'r') as secF:
    for line in secF:
        row_data = []
        for t in line.strip().split(" "):
            row_data.append(float(t))
        secure_outputs.append(torch.tensor(row_data).reshape(batch_size, 10))

print(secure_outputs[0].shape)

print(len(plaintext_outputs))
print(len(secure_outputs))

length = min(len(plaintext_outputs), len(secure_outputs))
# length = 0

for i in range(length):
    diff = plaintext_outputs[i].clone().detach() - secure_outputs[i]
    print(f'Iteration: {i}, max absoulte error: {torch.abs(diff).max()}')
    # print(f'Iteration: {i}, plain: {plaintext_outputs[i].clone().detach()[0]}, secure: {secure_outputs[i][0]}')
    where_diff = torch.abs(diff) > 1
    # where_diff = torch.isclose(plaintext_outputs[i].clone().detach(), secure_outputs[i], 1e-3)
    print(torch.masked_select(diff, where_diff))
    # print(torch.allclose(plaintext_outputs[i].clone().detach(), secure_outputs[i], atol=1e-2))
    # print(plaintext_outputs[i].clone().detach())
    # print(secure_outputs[i])

# print("Plaintext: ", plaintext_outputs[-1].clone().detach().numpy())

# for i, o in enumerate(secure_outputs):
#     print(f'Secure outputs-{i}: {o[0]}')
# print((plaintext_outputs[0].clone().detach()[0][0].numpy(), secure_outputs[0][0][0].numpy()))
# print(torch.allclose(plaintext_outputs[0].clone().detach()[0][0], secure_outputs[0][0][0], atol=1e-3))


torch.Size([32, 10])
torch.Size([32, 10])
1
800
Iteration: 0, max absoulte error: 7.525086402893066e-07
tensor([])


In [178]:
import torch
import os
# plaintext_outputs = torch.load(os.path.expanduser("~")+ "/DNN/output/plaintext-fc1-delta.pt").cpu()
# plaintext_outputs = torch.load(os.path.expanduser("~")+ "/DNN/output/plaintext-fc1-input.pt").cpu()
# relu1_input = torch.load(os.path.expanduser("~")+ "/DNN/output/plaintext-relu1-input.pt").cpu()
# relu1_output = torch.load(os.path.expanduser("~")+ "/DNN/output/plaintext-relu1-output.pt").cpu()
# plaintext_outputs = torch.load(os.path.expanduser("~")+ "/DNN/output/plaintext-cnn1-input.pt").cpu()
cnn1_output = torch.load(os.path.expanduser("~")+ "/DNN/output/plaintext-cnn1-output.pt").cpu()
# plaintext_outputs = torch.load(os.path.expanduser("~")+ "/DNN/output/plaintext-relu1-delta.pt").cpu()
# plaintext_outputs = torch.load(os.path.expanduser("~")+ "/DNN/output/plaintext-fc2-delta.pt").cpu()
# plaintext_outputs = torch.load(os.path.expanduser("~")+ "/DNN/output/plaintext-cnn5-delta.pt").cpu()
# plaintext_outputs = torch.load(os.path.expanduser("~")+ "/DNN/output/plaintext-cnn4-delta.pt").cpu()
# plaintext_outputs = torch.load(os.path.expanduser("~")+ "/DNN/output/plaintext-cnn2-delta.pt").cpu()
# plaintext_outputs = torch.load(os.path.expanduser("~")+ "/DNN/output/plaintext-bn1-bias_grad.pt").cpu()
# plaintext_outputs = torch.load(os.path.expanduser("~")+ "/DNN/output/plaintext-bn1-weight_grad.pt").cpu()
# plaintext_outputs = torch.load(os.path.expanduser("~")+ "/DNN/output/plaintext-bn2-bias_grad.pt").cpu()
# plaintext_outputs = torch.load(os.path.expanduser("~")+ "/DNN/output/plaintext-bn2-weight_grad.pt").cpu()
# plaintext_outputs = torch.load(os.path.expanduser("~")+ "/DNN/output/plaintext-mp1-delta.pt").cpu()
# plaintext_outputs = torch.load(os.path.expanduser("~")+ "/DNN/output/plaintext-bn1-delta.pt").cpu()
# plaintext_outputs = torch.load(os.path.expanduser("~")+ "/DNN/output/plaintext-bn2-delta.pt").cpu()
# plaintext_outputs = torch.load(os.path.expanduser("~")+ "/DNN/output/plaintext-cnn3-delta.pt").cpu()
# plaintext_outputs = torch.load(os.path.expanduser("~")+ "/DNN/output/plaintext-cnn3-input.pt").cpu()
# plaintext_outputs = torch.load(os.path.expanduser("~")+ "/DNN/output/plaintext-cnn2-input.pt").cpu()
# plaintext_outputs = torch.load(os.path.expanduser("~")+ "/DNN/output/plaintext-bn1-input.pt").cpu()
# plaintext_outputs = torch.load(os.path.expanduser("~")+ "/DNN/output/plaintext-bn1-output.pt").cpu()
# plaintext_outputs = torch.load(os.path.expanduser("~")+ "/DNN/output/plaintext-bn2-input.pt").cpu()
# plaintext_outputs = torch.load(os.path.expanduser("~")+ "/DNN/output/plaintext-bn2-output.pt").cpu()
# plaintext_outputs = torch.load(os.path.expanduser("~")+ "/DNN/output/plaintext-mp2-input.pt").cpu()
print(plaintext_outputs.shape)

iter = 16020
batch_size = 32

secure_outputs = []
# load secure training
with open(f"{os.path.expanduser('~')}/DNN/output/debug_delta.txt", 'r') as secF:
    for line in secF:
        row_data = []
        for t in line.strip().split(" "):
            row_data.append(float(t))
        secure_outputs.append(torch.tensor(row_data).reshape(plaintext_outputs.shape))

relu1_inputs_sec = []
# load secure training
with open(f"{os.path.expanduser('~')}/DNN/output/relu1_input.txt", 'r') as secF:
    for line in secF:
        row_data = []
        for t in line.strip().split(" "):
            row_data.append(float(t))
        relu1_inputs_sec.append(torch.tensor(row_data).reshape(batch_size, 256))
relu1_inputs_sec = relu1_inputs_sec[0]

relu1_outputs_sec = []
# load secure training
with open(f"{os.path.expanduser('~')}/DNN/output/relu1_output.txt", 'r') as secF:
    for line in secF:
        row_data = []
        for t in line.strip().split(" "):
            row_data.append(float(t))
        relu1_outputs_sec.append(torch.tensor(row_data).reshape(batch_size, 256))
relu1_outputs_sec = relu1_outputs_sec[0]

print(secure_outputs[0].shape)
secure_outputs = secure_outputs[0]
print(len(plaintext_outputs))
print(len(secure_outputs))

length = min(len(plaintext_outputs), len(secure_outputs))
# length = 0

for i in range(length):
    diff = plaintext_outputs[i].clone().detach() - secure_outputs[i]
    # diff = relu1_input[i].clone().detach() - relu1_inputs_sec[i]
    print(f'Iteration: {i}, max absoulte error: {torch.abs(diff).max()}')
    print(f'Iteration: {i}, plain: {plaintext_outputs[i].clone().detach()[:5]}, secure: {secure_outputs[i][:5]}')
    # print(f'Iteration: {i}, plain: {relu1_input[i].clone().detach()}, secure: {relu1_inputs_sec[i]}')
    where_diff = torch.abs(diff) > 1e-4
    # print(where_diff)
    # where_diff = torch.isclose(plaintext_outputs[i].clone().detach(), secure_outputs[i], 1e-3)

    # where_diff = relu1_input[i].clone().detach() * relu1_inputs_sec[i] < 0
    # print(torch.masked_select(relu1_input[i].clone().detach(), where_diff))
    # print(torch.masked_select(relu1_output[i].clone().detach(), where_diff))
    # print(torch.masked_select(relu1_inputs_sec[i], where_diff))
    # print(torch.masked_select(relu1_outputs_sec[i], where_diff))

    # print(torch.masked_select(plaintext_outputs[i].clone().detach(), where_diff))
    # print(torch.masked_select(secure_outputs[i], where_diff))
    # print(torch.allclose(plaintext_outputs[i].clone().detach(), secure_outputs[i], atol=1e-2))
    # print(plaintext_outputs[i].clone().detach())
    # print(secure_outputs[i])

# print("Plaintext: ", plaintext_outputs[-1].clone().detach().numpy())

# for i, o in enumerate(secure_outputs):
#     print(f'Secure outputs-{i}: {o[0]}')
# print((plaintext_outputs[0].clone().detach()[0][0].numpy(), secure_outputs[0][0][0].numpy()))
# print(torch.allclose(plaintext_outputs[0].clone().detach()[0][0], secure_outputs[0][0][0], atol=1e-3))


torch.Size([32, 96, 4, 4])
torch.Size([32, 96, 4, 4])
32
32
Iteration: 0, max absoulte error: 3.4928321838378906e-05
Iteration: 0, plain: tensor([[[-0.6180377007, -0.8604339361, -0.4643242359, -0.7417688966],
         [-0.9159026742, -0.7973853946, -0.2775729299, -0.2775729299],
         [-0.8896141648, -0.0273684319, -0.0273684319, -1.0072758198],
         [-0.8446184397, -0.0273684319, -0.0273684319, -0.8447254896]],

        [[-0.4499839246, -1.2440812588, -1.2440812588, -1.1294578314],
         [-0.6137093902, -1.3353147507, -1.3725885153, -1.0364984274],
         [-0.8166953921, -0.8166953921, -0.7261434197,  1.1693271399],
         [-0.7105318904,  0.3797387481,  0.6423028708,  0.6996605396]],

        [[ 1.2114617825, -0.3798493147, -0.5104625821, -0.6721163392],
         [ 0.0627886206, -0.1947851628, -0.1947851628, -0.6721163392],
         [-0.4882261753, -0.4882261753, -1.5636111498, -0.0821995512],
         [-0.6314764619, -1.3489533663, -0.6862720847, -0.0821995512]],

    

In [41]:
import torch
inpp = torch.tensor([ 0.0156,  0.0167, -0.0065,  0.0437,  0.0045, -0.0968,  0.0350,  0.0377,
         0.0503, -0.0388])

inpp2 = torch.tensor([ 0.0155,  0.0167, -0.0066,  0.0438,  0.0044, -0.0967,  0.0348,  0.0379,
         0.0504, -0.0387])
print(torch.nn.functional.softmax(inpp))
print(torch.nn.functional.softmax(inpp2))


tensor([0.1009, 0.1010, 0.0987, 0.1037, 0.0997, 0.0901, 0.1028, 0.1031, 0.1044,
        0.0955])
tensor([0.1008, 0.1010, 0.0986, 0.1037, 0.0997, 0.0901, 0.1028, 0.1031, 0.1044,
        0.0955])


/tmp/ipykernel_2187211/3257212514.py:7: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  print(torch.nn.functional.softmax(inpp))
/tmp/ipykernel_2187211/3257212514.py:8: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  print(torch.nn.functional.softmax(inpp2))
